In [1]:
liked_books = ["5981872", "17281240", "659600", "73086", "9637479", "227636"]

In [2]:
!head book_id_map.csv

book_id_csv,book_id
0,34684622
1,34536488
2,34017076
3,71730
4,30422361
5,33503613
6,33517540
7,34467031
8,6383669


In [30]:
csv_book_mapping = {}
with open("book_id_map.csv", "r") as f:
    while True:
        line = f.readline()
        if not line:
            break
        csv_id, book_id = line.strip().split(",")
        csv_book_mapping[csv_id] = book_id

In [31]:
len(csv_book_mapping)

2360651

In [32]:
!wc -l goodreads_interactions.csv

 228648343 goodreads_interactions.csv


In [33]:
!ls -lh | grep goodreaads_interactions.csv

In [34]:
!head goodreads_interactions.csv

user_id,book_id,is_read,rating,is_reviewed
0,948,1,5,0
0,947,1,5,1
0,946,1,5,0
0,945,1,5,0
0,944,1,5,0
0,943,1,5,0
0,942,1,5,0
0,941,1,5,0
0,940,1,5,0


In [35]:
overlap_users = set()

with open("goodreads_interactions.csv", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, csv_id, _, rating, _ = line.split(",")
        
        if user_id in overlap_users:
            continue
            
        try:
            rating = int(rating)
        except ValueError:
            continue
            
        book_id = csv_book_mapping[csv_id]
        
        if book_id in liked_books and rating >= 4:
            overlap_users.add(user_id)

In [36]:
# made a second loop so that I wouldnt miss all the rows before I found a book (in the user's read books) that was among my liked books 
rec_lines = [] # contains books that users who liked the same books as me have read
with open("goodreads_interactions.csv", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, csv_id, _, rating, _ = line.split(",")
        
        if user_id in overlap_users:
            book_id = csv_book_mapping[csv_id]
            rec_lines.append([user_id, book_id, rating])

In [41]:
len(overlap_users)

9314

In [42]:
len(rec_lines)

10433830

In [37]:
import pandas as pd

recs = pd.DataFrame(rec_lines, columns=["user_id", "book_id", "rating"])
recs["book_id"] = recs["book_id"].astype(str)

In [51]:
recs

,user_id,book_id,rating
0,120,104528,4
1,120,217086,3
2,120,1849,3
3,120,6526,4
4,120,1677996,5
...,...,...,...
10433825,875763,11227041,0
10433826,875763,73086,5
10433827,875763,11870085,0
10433828,875763,129613,0


In [45]:
top_recs = recs["book_id"].value_counts().head(10)
top_recs = top_recs.index.values

In [39]:
books_titles = pd.read_json("books_titles.json")
books_titles["book_id"] = books_titles["book_id"].astype(str)

In [40]:
books_titles.head()

,book_id,title,ratings,url,cover_image,mod_title
0,7327624,"The Unschooled Wizard (Sun Wolf and Starhawk, ...",140,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,the unschooled wizard sun wolf and starhawk 12
1,6066819,Best Friends Forever,51184,https://www.goodreads.com/book/show/6066819-be...,https://s.gr-assets.com/assets/nophoto/book/11...,best friends forever
2,287141,The Aeneid for Boys and Girls,46,https://www.goodreads.com/book/show/287141.The...,https://s.gr-assets.com/assets/nophoto/book/11...,the aeneid for boys and girls
3,6066812,All's Fairy in Love and War (Avalon: Web of Ma...,98,https://www.goodreads.com/book/show/6066812-al...,https://images.gr-assets.com/books/1316637798m...,alls fairy in love and war avalon web of magic 8
4,287149,The Devil's Notebook,986,https://www.goodreads.com/book/show/287149.The...,https://images.gr-assets.com/books/1328768789m...,the devils notebook


In [46]:
books_titles[books_titles["book_id"].isin(top_recs)]

,book_id,title,ratings,url,cover_image,mod_title
156884,10818853,"Fifty Shades of Grey (Fifty Shades, #1)",1348198,https://www.goodreads.com/book/show/10818853-f...,https://images.gr-assets.com/books/1385207843m...,fifty shades of grey fifty shades 1
183171,9637479,"Dragon Bound (Elder Races, #1)",30605,https://www.goodreads.com/book/show/9637479-dr...,https://images.gr-assets.com/books/1312822853m...,dragon bound elder races 1
284473,2767052,"The Hunger Games (The Hunger Games, #1)",4899965,https://www.goodreads.com/book/show/2767052-th...,https://images.gr-assets.com/books/1447303603m...,the hunger games the hunger games 1
323031,41865,"Twilight (Twilight, #1)",3941381,https://www.goodreads.com/book/show/41865.Twil...,https://images.gr-assets.com/books/1361039443m...,twilight twilight 1
352861,49041,"New Moon (Twilight, #2)",1161751,https://www.goodreads.com/book/show/49041.New_...,https://images.gr-assets.com/books/1361039440m...,new moon twilight 2
403188,1162543,"Breaking Dawn (Twilight, #4)",1078310,https://www.goodreads.com/book/show/1162543.Br...,https://images.gr-assets.com/books/1361039438m...,breaking dawn twilight 4
415350,11505797,"Beautiful Disaster (Beautiful, #1)",425898,https://www.goodreads.com/book/show/11505797-b...,https://images.gr-assets.com/books/1358259032m...,beautiful disaster beautiful 1
878545,3,Harry Potter and the Sorcerer's Stone (Harry P...,4765497,https://www.goodreads.com/book/show/3.Harry_Po...,https://images.gr-assets.com/books/1474154022m...,harry potter and the sorcerers stone harry pot...
952424,428263,"Eclipse (Twilight, #3)",1146155,https://www.goodreads.com/book/show/428263.Ecl...,https://images.gr-assets.com/books/1361038355m...,eclipse twilight 3
1064278,42899,"Dark Lover (Black Dagger Brotherhood, #1)",231228,https://www.goodreads.com/book/show/42899.Dark...,https://images.gr-assets.com/books/1429676285m...,dark lover black dagger brotherhood 1


In [49]:
all_recs = recs["book_id"].value_counts()

In [50]:
all_recs

book_id
41865       5709
2767052     4852
42899       4647
10818853    4428
3           4276
            ... 
17318195       1
10139649       1
6937258        1
56545          1
17623735       1
Name: count, Length: 781143, dtype: int64

In [54]:
all_recs = all_recs.to_frame().reset_index()

In [55]:
all_recs

,book_id,count
0,41865,5709
1,2767052,4852
2,42899,4647
3,10818853,4428
4,3,4276
...,...,...
781138,17318195,1
781139,10139649,1
781140,6937258,1
781141,56545,1


In [56]:
all_recs = all_recs.merge(books_titles, how="inner", on="book_id")

In [57]:
all_recs

,book_id,count,title,ratings,url,cover_image,mod_title
0,41865,5709,"Twilight (Twilight, #1)",3941381,https://www.goodreads.com/book/show/41865.Twil...,https://images.gr-assets.com/books/1361039443m...,twilight twilight 1
1,2767052,4852,"The Hunger Games (The Hunger Games, #1)",4899965,https://www.goodreads.com/book/show/2767052-th...,https://images.gr-assets.com/books/1447303603m...,the hunger games the hunger games 1
2,42899,4647,"Dark Lover (Black Dagger Brotherhood, #1)",231228,https://www.goodreads.com/book/show/42899.Dark...,https://images.gr-assets.com/books/1429676285m...,dark lover black dagger brotherhood 1
3,10818853,4428,"Fifty Shades of Grey (Fifty Shades, #1)",1348198,https://www.goodreads.com/book/show/10818853-f...,https://images.gr-assets.com/books/1385207843m...,fifty shades of grey fifty shades 1
4,3,4276,Harry Potter and the Sorcerer's Stone (Harry P...,4765497,https://www.goodreads.com/book/show/3.Harry_Po...,https://images.gr-assets.com/books/1474154022m...,harry potter and the sorcerers stone harry pot...
...,...,...,...,...,...,...,...
660012,17318195,1,The Complete Grimm's Fairy Tales,55,https://www.goodreads.com/book/show/17318195-t...,https://images.gr-assets.com/books/1384382536m...,the complete grimms fairy tales
660013,10139649,1,Idea Man,2171,https://www.goodreads.com/book/show/10139649-i...,https://images.gr-assets.com/books/1303177089m...,idea man
660014,6937258,1,"Return to the Little Kingdom: Steve Jobs, the ...",506,https://www.goodreads.com/book/show/6937258-re...,https://s.gr-assets.com/assets/nophoto/book/11...,return to the little kingdom steve jobs the cr...
660015,56545,1,The Second Coming of Steve Jobs,1940,https://www.goodreads.com/book/show/56545.The_...,https://s.gr-assets.com/assets/nophoto/book/11...,the second coming of steve jobs


In [59]:
all_recs["score"] = all_recs["count"] * (all_recs["count"] / all_recs["ratings"])

In [60]:
all_recs.sort_values("score", ascending=False).head(10)

,book_id,count,title,ratings,url,cover_image,mod_title,score
5878,34379276,262,"Hello Stranger (The Ravenels, #4)",31,https://www.goodreads.com/book/show/34379276-h...,https://images.gr-assets.com/books/1505345361m...,hello stranger the ravenels 4,2214.322581
6548,35463752,241,"Lake Silence (The Others, #6)",30,https://www.goodreads.com/book/show/35463752-l...,https://images.gr-assets.com/books/1497821052m...,lake silence the others 6,1936.033333
1683,17333180,641,"Magic Triumphs (Kate Daniels, #10)",219,https://www.goodreads.com/book/show/17333180-m...,https://images.gr-assets.com/books/1502893641m...,magic triumphs kate daniels 10,1876.168950
5541,30242230,275,Marriage of Inconvenience (Knitting in the Cit...,41,https://www.goodreads.com/book/show/30242230-m...,https://images.gr-assets.com/books/1471385814m...,marriage of inconvenience knitting in the city 7,1844.512195
2831,35294265,457,"Burn Bright (Alpha & Omega, #5)",134,https://www.goodreads.com/book/show/35294265-b...,https://images.gr-assets.com/books/1504296120m...,burn bright alpha omega 5,1558.574627
8055,30315676,202,"Dr. Strange Beard (Winston Brothers, #5)",28,https://www.goodreads.com/book/show/30315676-d...,https://images.gr-assets.com/books/1505833129m...,dr strange beard winston brothers 5,1457.285714
7320,31681941,219,The Comfort Zone,36,https://www.goodreads.com/book/show/31681941-t...,https://s.gr-assets.com/assets/nophoto/book/11...,the comfort zone,1332.250000
2485,35457273,501,"High Voltage (Fever, #10)",189,https://www.goodreads.com/book/show/35457273-h...,https://images.gr-assets.com/books/1503486372m...,high voltage fever 10,1328.047619
10474,30111415,162,"The Ippos King (Wraith Kings, #3)",20,https://www.goodreads.com/book/show/30111415-t...,https://images.gr-assets.com/books/1469348434m...,the ippos king wraith kings 3,1312.200000
5614,25026912,272,"Untitled (Mercy Thompson, #11)",60,https://www.goodreads.com/book/show/25026912-u...,https://s.gr-assets.com/assets/nophoto/book/11...,untitled mercy thompson 11,1233.066667


In [61]:
popular_recs = all_recs[all_recs["count"] > 75].sort_values("score", ascending=False)

In [62]:
def make_clickable(val):
    return '<a target="_blank" href="{}">Goodreads</a>'.format(val)

def show_image(val):
    return '<img src="{}" width=50></img'.format(val)

popular_recs[~popular_recs["book_id"].isin(liked_books)].head(10).style.format({'url': make_clickable, 'cover_image': show_image})

,book_id,count,title,ratings,url,cover_image,mod_title,score
5878,34379276,262,"Hello Stranger (The Ravenels, #4)",31,Goodreads,</img,hello stranger the ravenels 4,2214.322581
6548,35463752,241,"Lake Silence (The Others, #6)",30,Goodreads,</img,lake silence the others 6,1936.033333
1683,17333180,641,"Magic Triumphs (Kate Daniels, #10)",219,Goodreads,</img,magic triumphs kate daniels 10,1876.168950
5541,30242230,275,"Marriage of Inconvenience (Knitting in the City, #7)",41,Goodreads,</img,marriage of inconvenience knitting in the city 7,1844.512195
2831,35294265,457,"Burn Bright (Alpha & Omega, #5)",134,Goodreads,</img,burn bright alpha omega 5,1558.574627
8055,30315676,202,"Dr. Strange Beard (Winston Brothers, #5)",28,Goodreads,</img,dr strange beard winston brothers 5,1457.285714
7320,31681941,219,The Comfort Zone,36,Goodreads,</img,the comfort zone,1332.250000
2485,35457273,501,"High Voltage (Fever, #10)",189,Goodreads,</img,high voltage fever 10,1328.047619
10474,30111415,162,"The Ippos King (Wraith Kings, #3)",20,Goodreads,</img,the ippos king wraith kings 3,1312.200000
5614,25026912,272,"Untitled (Mercy Thompson, #11)",60,Goodreads,</img,untitled mercy thompson 11,1233.066667
